# Test Strategies against S&P 500 Baseline


In [2]:
import sys
sys.executable

'/home/hilton/Coding/Dashboard/backtest-env/bin/python'

# Section 0 - Config and Imports

In [3]:
import numpy as np
import pandas as pd
import datetime
import yfinance as yf

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from scipy import stats
from typing import Dict, Callable, Optional


import warnings
warnings.filterwarnings("ignore")


# -------------------------
# Global constants
# -------------------------
TRADING_DAYS = 252
RISK_FREE_RATE = 0.0

PLOT_STYLE = {
    "figure.figsize": (14, 6),
    "axes.grid": True,
    "grid.alpha": 0.3,
    "axes.spines.top": False,
    "axes.spines.right": False
}

plt.rcParams.update(PLOT_STYLE)

In [ ]:
def extractTicker(tickerNames):
    errors = {}
    data = {}
    for ticker in tickerNames:
        df = yf.download(ticker, period="25y", progress=False)

        if df.empty or df.filter(like="Close").isna().all().all():
            errors[ticker] = "invalid ticker or no usable data"
            continue
        
        startDate = df.index.min()
        RequiredstartDate = pd.Timestamp.today() - pd.DateOffset(years=2)

        if startDate > RequiredstartDate:
            errors[ticker] = "insufficient history"
            continue

        data[ticker] = {
            "prices": df
        }

    return data, errors


In [ ]:
def extractFundementals(tickerNames):
    f_errors = {}
    fundementals = []
    
    asof = pd.Timestamp.today()

    for ticker in tickerNames:
        try:
            info = yf.Ticker(ticker).info
        except Exception as e:
            f_errors[ticker] = f"fetch info failed: {e}"
            continue
    
    PE = info.get("trailingPE", np.nan)
    Market_Cap = info.get("marketCap", np.nan)
    BookValue_PerShare = info.get("bookValue", np.nan)
    Intangible = info.get("intangibleAssets", np.nan)
    Shares_Outstanding = info.get("sharesOutstanding", np.nan)

    if(pd.notna(BookValue_PerShare)
       and pd.notna(Shares_Outstanding)
       and pd.notna(Market_Cap)):
        Book = float(BookValue_PerShare) * float(Shares_Outstanding)
        BooktoMarket = Book / float(Market_Cap)
    else:
        Book, BooktoMarket = np.nan, np.nan

    if (pd.notna(Book)
        and pd.notna(Intangible)):
        tangibleBook = Book - float(Intangible)
        tangibleBooktoMarket = tangibleBook/float(Market_Cap)
    else: 
        tangibleBook, tangibleBooktoMarket = np.nan, np.nan

    fundementals.append({
        "ticker": ticker,
        "PE": PE,
        "Market_Cap": Market_Cap,
        "BookValue_PerShare": BookValue_PerShare,
        "Intangible": Intangible,
        "Shares_Outstanding": Shares_Outstanding,
        "Book": Book,
        "BooktoMarket": BooktoMarket,
        "Tangible Book": tangibleBook,
        "TangibleBooktoMarket": tangibleBooktoMarket
    })

    fundementals_df = pd.DataFrame(fundementals).set_index("ticker")
    return fundementals_df, f_errors    

    

In [34]:
tickerNames = ["^GSPC"]

data, errors = extractTicker(tickerNames)
fundementals, f_errors = extractFundementals(tickerNames)

TypeError: cannot unpack non-iterable float object

# Section 1 - Base Feature Engineering

### Daily Return

$$
r_t = \frac{C_t}{C_{t-1}} - 1
$$


In [ ]:
df = data["^GSPC"]
df["Daily Return"] = df["Close"].pct_change()
df = df.dropna()

### Equity Curve

$$
E_t = \prod_{i=1}^{t} \left(1 + r_i\right)
$$
$$
E_0 = 1
$$



In [ ]:
df["Equity Curve"] = (1 + df["Daily Return"]).cumprod()

### Rolling Peak Equity -> Drawdown Series -> Max Drawdown

$$
M_t = \max_{s \le t} E_s
$$

$$
\text{DD}_t = \frac{E_t}{M_t} - 1
$$
$$
\text{Max Drawdown} = \min_t \left( \text{DD}_t \right)
$$


In [8]:
df["rolling_max"] = df["Equity Curve"].cummax()

df["Drawdown Series"] = df["Equity Curve"]/df["rolling_max"] - 1

max_drawdown = df["Drawdown Series"].min()

df[["rolling_max", "Drawdown Series"]]

Price,rolling_max,Drawdown Series
Ticker,,
Date,,
2001-01-03,1.050099,0.000000
2001-01-04,1.050099,-0.010552
2001-01-05,1.050099,-0.036518
2001-01-08,1.050099,-0.038366
2001-01-09,1.050099,-0.034700
...,...,...
2025-12-22,5.377668,-0.003262
2025-12-23,5.384518,0.000000


### Drawdown Duration

$$
D_t =
\begin{cases}
0, & \text{if } E_t = \max_{s \le t} E_s \\
D_{t-1} + 1, & \text{if } E_t < \max_{s \le t} E_s
\end{cases}
$$


In [27]:
dd = df["Drawdown Series"].to_numpy()
duration = np.zeros(len(df), dtype=int)

for i in range(1, len(df)):
    duration[i] = duration[i-1] + 1 if dd[i] < 0 else 0

df["Drawdown Duration"] = duration
max_drawdown_duration = duration.max()
max_drawdown_duration


np.int64(1439)

### Moving Average

$$
\text{MA}_{N,t} = \frac{1}{N} \sum_{i=0}^{N-1} C_{t-i}
$$


In [28]:
df["MA_200"] = df["Close"].rolling(200).mean()

# Section 2 - Basic Strategy Implementation & Comparison

### Strategy 1 - Naive Price Theoretical Investing 

#### Conditions:

#### P/E ratio less than 7 and/or Tangible Book / Market > 1

#### Market Cap under 1 billion 
